# <center> <font color='#0B5345'> <b> Customized callbacks </font> </b> </center>

### <b> <font color='blue'> Table of Contents </b> </font>

- 0. [Introduction](#0)
- 1. [Libraries](#1)
- 2. [Loading data](#2)
- 3. [Model Building](#3)
- 4. [Training](#4)


<a name="0"></a>
## <b> <font color='##138D75'> 0. Introduction </b> </font>


A callback in TensorFlow is a set of functions that are executed at specific stages during the training process of a neural network model. Callbacks allow you to customize and extend the behavior of the training process, enabling you to perform additional tasks such as logging, model checkpointing, learning rate scheduling, and early stopping.

You can see the methods below:



<table style="border: 1px solid;">
  <tr style="border: 1px solid;">
      <th style="border: 1px solid;"><center>Method</center></th>
      <th><center>Description</center></th>
  </tr>
  <tr style="border: 1px solid;">
    <td style="border: 1px solid;">on_epoch_begin(epoch, logs=None)</td>
    <td style="text-align: left;">Called at the beginning of each epoch during training. <br> Allows for pre-epoch tasks or updates.</td>
  </tr>
  <tr style="border: 1px solid;">
    <td style="border: 1px solid;">on_epoch_end(epoch, logs=None)</td>
    <td style="text-align: left;">Called at the end of each epoch during training. <br>Enables logging, learning rate updates, or early stopping based on epoch results.</td>
  </tr>
  <tr style="border: 1px solid;">
    <td style="border: 1px solid;">on_batch_begin(batch, logs=None)</td>
    <td style="text-align: left;">Called at the beginning of each batch during training. <br> Allows for pre-batch operations such as data preprocessing.</td>
  </tr>
  <tr style="border: 1px solid;">
    <td style="border: 1px solid;">on_batch_end(batch, logs=None)</td>
    <td style="text-align: left;">Called at the end of each batch during training. <br> Enables logging batch metrics or updating training progress.</td>
  </tr>
  <tr style="border: 1px solid;">
    <td style="border: 1px solid;">on_train_begin(logs=None)</td>
    <td style="text-align: left;">Called at the beginning of the training process. <br> Allows for initialization tasks or setup procedures before training.</td>
  </tr>
  <tr style="border: 1px solid;">
    <td style="border: 1px solid;">on_train_end(logs=None)</td>
    <td style="text-align: left;">Called at the end of the training process. <br> Enables cleanup tasks or finalization operations after training completes.</td>
  </tr>
</table>




<a name="1"></a>
## <b> <font color='##138D75'> 1. Libraries </b> </font>

In [6]:
# que no se impriman info y warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

# Ignore all warnings
warnings.filterwarnings('ignore')

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, callbacks, models


<a name="2"></a>
## <b> <font color='##138D75'> 2. Loading data </b> </font>

In [9]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist

# Load data into training and testing sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0


<a name="3"></a>
## <b> <font color='##138D75'> 3. Model building </b> </font>

In [15]:
def build_model():

    # Define the model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax'),
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
        
    return model

<a name="4"></a>
## <b> <font color='##138D75'> 4. Training </b> </font>

In [19]:
# let's define a customized callback, that stops the training when it reaches 97% accuracy
class myCallback(tf.keras.callbacks.Callback):
        # Define the correct function signature for on_epoch_end
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') is not None and logs.get('accuracy') > 0.97:                 
                print("\nReached 97% accuracy so cancelling training!")
                # Stop training once the above condition is met
                self.model.stop_training = True

In [20]:
# model
model = build_model()

# callback
callbacks = myCallback()

# fit the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test,y_test),callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 26s 13ms/step - loss: 0.1540 - accuracy: 0.9546 - val_loss: 0.0596 - val_accuracy: 0.9797
Epoch 2/10
1874/1875 [============================>.] - ETA: 0s - loss: 0.0522 - accuracy: 0.9843
Reached 97% accuracy so cancelling training!
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0521 - accuracy: 0.9843 - val_loss: 0.0487 - val_accuracy: 0.9832


We are going to enhance our callback by passing the target accuracy as an argument instead of hardcoding it.

In [21]:
class myCallback(tf.keras.callbacks.Callback):
    def __init__(self, target_accuracy):
        super(myCallback, self).__init__()
        self.target_accuracy = target_accuracy

    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') is not None and logs.get('accuracy') > self.target_accuracy:
            print(f"\nReached {self.target_accuracy * 100}% accuracy, so cancelling training!")
            self.model.stop_training = True


In [23]:
# model
model = build_model()

# callback
callback = myCallback(target_accuracy=0.99)

# fit the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test,y_test),callbacks=[callback])

Epoch 1/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.1505 - accuracy: 0.9546 - val_loss: 0.0567 - val_accuracy: 0.9812
Epoch 2/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0518 - accuracy: 0.9844 - val_loss: 0.0570 - val_accuracy: 0.9817
Epoch 3/10
1875/1875 [==============================] - 25s 14ms/step - loss: 0.0344 - accuracy: 0.9896 - val_loss: 0.0548 - val_accuracy: 0.9829
Epoch 4/10
1873/1875 [============================>.] - ETA: 0s - loss: 0.0221 - accuracy: 0.9932
Reached 99.0% accuracy, so cancelling training!
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0221 - accuracy: 0.9932 - val_loss: 0.0430 - val_accuracy: 0.9859
